<a href="https://colab.research.google.com/github/Robert-Gomez-AI/Generative_models/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers